In [ ]:
import larix as lx
import jax
import jax.numpy as jnp
from pytest import approx

In [ ]:
d = lx.examples.MTC(format='dataset')

In [ ]:
m = lx.Model(d)
from larch import P, X, PX
m.utility_co[2] = P("ASC_SR2")  + P("hhinc#2") * X("hhinc")
m.utility_co[3] = P("ASC_SR3P") + P("hhinc#3") * X("hhinc")
m.utility_co[4] = P("ASC_TRAN") + P("hhinc#4") * X("hhinc")
m.utility_co[5] = P("ASC_BIKE") + P("hhinc#5") * X("hhinc")
m.utility_co[6] = P("ASC_WALK") + P("hhinc#6") * X("hhinc")

m.utility_ca = PX("tottime") + PX("totcost")

m.availability_var = 'avail'
m.choice_ca_var = 'chose'

m.title = "MTC Example 1 (Simple MNL)"
r = m.maximize_loglike(quiet=True)

In [ ]:
m.jax_loglike(m.pvals) # 622ms

In [ ]:
assert m.jax_loglike(m.pvals) == approx(-3626.186)

In [ ]:
m.jax_d_loglike(m.pvals) # 1.69s

In [ ]:
assert m.d_loglike(r.x/2) == approx([  
    -100.668873,     -427.881921,   -346.047421,   
    -176.396743,      -69.950285, -25639.434015, 
    -20581.626877, -10752.088732,  -5810.403543,  
    -4360.946795,   54115.923707, -16552.567557,
])

In [ ]:
assert m.jax_d_loglike(np.asarray(r.x/2)) == approx([  
    -100.668873,     -427.881921,   -346.047421,   
    -176.396743,      -69.950285, -25639.434015, 
    -20581.626877, -10752.088732,  -5810.403543,  
    -4360.946795,   54115.923707, -16552.567557,
], rel=1e-5)

In [ ]:
m.pvals = 'null'

In [ ]:
assert all(m.pf['value'] == 0)

In [ ]:
rj = m.jax_maximize_loglike() # 263ms

In [ ]:
rj

In [ ]:
assert rj.loglike == approx(-3626.18603515625)

In [ ]:
h = m.jax_d2_loglike(m.pvals) # 4.00s

In [ ]:
np.sqrt(np.linalg.pinv(h).diagonal())

In [ ]:
ihess = m.jax_invhess_loglike(m.pvals) # 2.65s

In [ ]:
se, hess, ihess = m.jax_param_cov(m.pvals) # 199ms

In [ ]:
assert m.pstderr == approx(np.array([ 3.045759e-01,  1.046350e-01,  1.776854e-01,  1.325887e-01,  1.941035e-01,  1.553271e-03,  2.537731e-03,
        1.828749e-03,  5.326436e-03,  3.033601e-03,  2.388836e-04,  3.099270e-03], dtype=np.float32))